<a href="https://colab.research.google.com/github/japjotsaggu/multilayer_perceptron/blob/main/MultiLayerPerceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#multi layer perceptron - fully connected neural netwrok

In [2]:
import random
import math

In [17]:
class Neuron():
  def __init__(self, position_in_layer, is_output_neuron = False):
    self.weights = []
    self.input = []
    self.output = None

    #backprop update
    self.updated_weights = []
    #how to update weights
    self.is_output_neuron = is_output_neuron
    #delta for update at backpropogation
    self.delta = None
    #used for updating in backprop
    self.position_in_layer = position_in_layer

  def attach_to_output(self, neurons):
    '''
       Helper function to store the reference of the other neurons
       to this particular neuron (used for backpropagation)
    '''
    self.output_neurons = neurons

  def sigmoid(self, x):
    '''
       sigmoid function for activation
    '''
    return 1/ (1 + math.exp(-x))

  def init_weights(self, num_input):
    '''
       when we know the num of inputs for a neuron, this is to setup weights
    '''
    #randomly initialize the weights
    for i in range(num_input+1):
      self.weights.append(random.uniform(0, 1))

  def predict(self, row):
    #reset the inputs
    self.inputs = []
    #iterating over the weights and the feautres in a given row
    activation = 0
    for weight, feature in zip(self.weights, row):
      self.inputs.append(feature)
      activation = activation + weight*feature

    self.output = self.sigmoid(activation)
    return self.output


  def update_neuron(self):
    '''
       updating weights at the end of backpropogation
    '''
    self.weights = []
    for new_weight in self.updated_weights:
      self.weights.append(new_weight)

  def calculate_update(self, learning_rate, target):
    '''
       Calculating updated weights for this neuron.
       First calculate the right delta, then calculate the right updated weights.
       Not overwriting the weights as yet as they are needed for other update in backprop algo
    '''
    if self.is_output_neuron:
      #calculating delta
      self.delta = (self.output - target)*self.output*(1-self.output)

    else:
      delta_sum = 0
      #to know which weights this neuron is contributing in the output layer
      curr_weight_index = self.position_in_layer
      for output_neuron in self.output_neurons:
        delta_sum = delta_sum + (output_neuron.delta * output_neuron.weights[curr_weight_index])

      #update this neuron delta
      self.delta = delta_sum*self.output*(1-self.output)


    self.updated_weights = []

    #iterate over each weight and update
    for curr_weight, curr_input in zip(self.weights, self.inputs):
      gradient = self.delta*curr_input
      new_weight = curr_weight - learning_rate*gradient
      self.updated_weights.append(new_weight)



In [4]:
class Layer():
  '''
     a layer in a fully connected feed forward neural network
  '''

  def __init__(self, num_neuron, is_output_layer = False):
    self.is_output_layer = is_output_layer
    self.neurons = []
    #creating neurons
    for i in range(num_neuron):
      neuron = Neuron(i, is_output_neuron = is_output_layer)
      self.neurons.append(neuron)

  def attach(self, layer):
    '''
        attaching neurons from this layer to another one
        needed for backrprop algorithm
    '''
    for in_neuron in self.neurons:
      in_neuron.attach_to_output(layer.neurons)

  def init_layer(self, num_input):
    '''
       initializing the weights of each neuron in the layer
    '''
    for neuron in self.neurons:
      neuron.init_weights(num_input)

  def predict(self, row):
    '''
       calculating activation for the full layer given the row of data streaming in
    '''
    row.append(1) #bias
    activations = [neuron.predict(row) for neuron in self.neurons]
    return activations

In [11]:
class MultiLayerPerceptron():
  '''
     an input layer, a perceptrons layer and a one neuron output layer which does binary classification
  '''
  def __init__(self, learning_rate = 0.01, num_iterations = 100):
    self.layers = []
    self.learning_rate = learning_rate
    self.num_iterations = num_iterations

  def add_output_layer(self, num_neuron):
    '''
       creating a new output layer and adding it to the architecture
    '''
    self.layers.insert(0, Layer(num_neuron, is_output_layer = True))

  def add_hidden_layer(self, num_neuron):
    '''
      create a hidden layer, add it to the architecture and finally attach it to the front of the architecture
    '''
    hidden_layer = Layer(num_neuron)
    hidden_layer.attach(self.layers[0])
    self.layers.insert(0, hidden_layer)

  def update_layers(self, target):
    '''
        calculating updated weights & then updating the weights all at once
    '''
    for layer in reversed(self.layers):
      for neuron in layer.neurons:
        neuron.calculate_update(self.learning_rate, target)

    for layer in self.layers:
      for neuron in layer.neurons:
        neuron.update_neuron()

  def fit(self, X, y):
    '''
        main training function of nn algorithm.
        will make use of backprop
        stochastic gradient descent by selecting one row at random from the dataset
        and use predict to calculate the error
        the error is then backpropogated and new weights calculated
        whole network weights updated
    '''
    num_row = len(X)
    num_feature = len(X[0])

    #init the weights throughout each of the layer
    self.layers[0].init_layer(num_feature)

    for i in range(1, len(self.layers)):
      num_input = len(self.layers[i-1].neurons)
      self.layers[i].init_layer(num_input)

    for i in range(self.num_iterations):
      r_i = random.randint(0, num_row -1)
      row = X[r_i]
      yhat = self.predict(row)
      target = y[r_i]

      self.update_layers(target)

      #calculating error at every 100 iterations
      if i%1000 == 0:
        total_error = 0
        for r_i in range(num_row):
          row = X[r_i]
          yhat = self.predict(row)
          error = (y[r_i] - yhat)
          total_error = total_error + error**2
        mean_error = total_error/num_row

  def predict(self, row):
    '''
        takes a row of input and returns the output of the whole neural network
    '''
    activations = self.layers[0].predict(row)
    for i in range(1, len(self.layers)):
      activations = self.layers[i].predict(activations)

    outputs = []
    for activation in activations:
      if activation >= 0.5:
        outputs.append(1.0)

      else:
        outputs.append(0.0)

    return outputs[0]


In [21]:
# XOR function (to test the nn)
X = [[0,0], [0,1], [1,0], [1,1]]
y = [0, 1, 1, 0]

clf = MultiLayerPerceptron(learning_rate = 0.1, num_iterations = 100000)
# Create the architecture
clf.add_output_layer(num_neuron = 1)
clf.add_hidden_layer(num_neuron = 3)
clf.add_hidden_layer(num_neuron = 2)


clf.fit(X,y)

In [22]:
clf.predict([0,0])

0.0

In [23]:
clf.predict([1,0])

1.0